<a href="https://colab.research.google.com/github/germancruzram/Frame-marking/blob/main/VF_pkm_rotulo_y_GIS_shared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PYTHON PARA UN PROYECTO VIAL♠**  

Ing. German Cruz R.🇳


**INSTALANDO LAS LIBRERIAS A UTILIZAR**

In [4]:
!pip install exifread
!pip install piexif
!!pip install geopy
!pip install rarfile
!pip install Pillow piexif



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.1 MB/s eta 0:00:00


**CONFIGURANDO CARPETAS DE TRABAJO**

In [5]:
import os


folder_paths = ["/content/imagenes", "/content/imagenes_con_telemetria"]

for folder_path in folder_paths:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"La carpeta {folder_path} ha sido creada con éxito.")
    else:
        print(f"La carpeta {folder_path} ya existe.")
    !ls


La carpeta /content/imagenes ha sido creada con éxito.
imagenes  sample_data
La carpeta /content/imagenes_con_telemetria ha sido creada con éxito.
imagenes  imagenes_con_telemetria  sample_data


**AJUSTANDO RESOLUCION DE LAS IMAGENES**

In [6]:
import os
from PIL import Image, ImageOps
import piexif


imagenes_dir = '/content/imagenes'

def resize_image(file_path):
    image = Image.open(file_path)
    exif_data = piexif.load(image.info['exif']) if 'exif' in image.info else None
    new_width = 1280
    new_height = 720
    image_resized = image.resize((new_width, new_height), Image.LANCZOS)

    if exif_data:
        exif_bytes = piexif.dump(exif_data)
        image_resized.save(file_path, exif=exif_bytes)
    else:
        image_resized.save(file_path)

for filename in os.listdir(imagenes_dir):
    file_path = os.path.join(imagenes_dir, filename)
    if os.path.isfile(file_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff')):
        resize_image(file_path)


**ORDENANDO LAS FOTOS SECUENCIALMENTE**

In [10]:
import os
import piexif

def get_image_datetime(image_path):
    """
    Obtiene la fecha y hora de captura de una imagen utilizando la información Exif.
    Args:
        image_path (str): La ruta de la imagen.
    Returns:
        str: La fecha y hora de captura en formato de cadena si está disponible,
             una fecha mínima si no hay información EXIF.
    """
    try:
        exif_data = piexif.load(image_path)
        date_time_original = exif_data["Exif"][piexif.ExifIFD.DateTimeOriginal]
        return date_time_original.decode('utf-8')
    except (KeyError, AttributeError):
        return "0000:00:00 00:00:00"  # Fecha mínima si no hay información EXIF

def rename_images_by_image_datetime(folder_path):
    """
    Renombra las imágenes en la carpeta especificada basándose en la fecha y hora de captura.
    Las imágenes se renombran de forma consecutiva según la hora de captura, desde la más temprana a la más tardía.
    Se preserva la información EXIF original.
    Args:
        folder_path (str): La ruta de la carpeta que contiene las imágenes.
    """
    files = os.listdir(folder_path)
    image_files = [file for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg'))]
    image_files.sort(key=lambda x: get_image_datetime(os.path.join(folder_path, x)))

    count = 1

    for image_name in image_files:
        # Construir la ruta completa de la imagen antigua y nueva
        old_image_path = os.path.join(folder_path, image_name)
        new_image_name = f"{count:02d}.jpg"
        new_image_path = os.path.join(folder_path, new_image_name)

        os.rename(old_image_path, new_image_path)
        count += 1
        try:
            exif_data = piexif.load(old_image_path)
            exif_bytes = piexif.dump(exif_data)
            piexif.insert(exif_bytes, new_image_path)
        except Exception as e:
            print(f" ")


folder_path = "/content/imagenes"

rename_images_by_image_datetime(folder_path)

**LISTANDO IMAGENES CON COORDENADAS Y CALCULO DE DISTANCIAS 3D**

In [ ]:
import os
import exifread
import pandas as pd
import math

def get_decimal_degrees(dms, ref):
    degrees = dms.values[0].num / dms.values[0].den
    minutes = dms.values[1].num / dms.values[1].den
    seconds = dms.values[2].num / dms.values[2].den
    decimal = degrees + (minutes / 60.0) + (seconds / 3600.0)
    if ref in ['S', 'W']:
        decimal *= -1
    return decimal

def extract_image_info(image_path, prev_coordinates=None, accumulated_distance=0):
    with open(image_path, 'rb') as f:
        tags = exifread.process_file(f)
        if not tags:
            return None, None, None, None, None, 0, accumulated_distance

        latitude = tags.get('GPS GPSLatitude')
        longitude = tags.get('GPS GPSLongitude')
        latitude_ref = tags.get('GPS GPSLatitudeRef')
        longitude_ref = tags.get('GPS GPSLongitudeRef')
        altitude = tags.get('GPS GPSAltitude')
        date_time = tags.get('EXIF DateTimeOriginal')

        if latitude and longitude and latitude_ref and longitude_ref:
            latitude_dec = get_decimal_degrees(latitude, latitude_ref.printable)
            longitude_dec = get_decimal_degrees(longitude, longitude_ref.printable)
        else:
            latitude_dec, longitude_dec = None, None

        altitude_val = altitude.values[0].num / altitude.values[0].den if altitude else None
        date_time_str = str(date_time) if date_time else None

        distance = 0
        if prev_coordinates and latitude_dec is not None and longitude_dec is not None:
            if altitude_val is not None:

                lat1 = math.radians(prev_coordinates[0])
                lon1 = math.radians(prev_coordinates[1])
                lat2 = math.radians(latitude_dec)
                lon2 = math.radians(longitude_dec)

                R = 6371.0 * 1000
                d_altitude = abs(altitude_val - prev_coordinates[2])
                dlat = lat2 - lat1
                dlon = lon2 - lon1

                a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
                c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
                distance_horizontal = R * c
                distance = math.sqrt(distance_horizontal ** 2 + d_altitude ** 2) / 1000  # Convertir a kilómetros

        accumulated_distance += distance

        return (os.path.basename(image_path), latitude_dec, longitude_dec, altitude_val, date_time_str, distance, round(accumulated_distance, 4))

def create_excel_with_image_info(folder_path, excel_path, initial_accumulated_distance=0):
    image_info_list = []
    prev_coordinates = None
    accumulated_distance = initial_accumulated_distance

    for filename in sorted(os.listdir(folder_path)):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(folder_path, filename)
            info = extract_image_info(image_path, prev_coordinates, accumulated_distance)
            if info[1] is not None and info[2] is not None:
                prev_coordinates = (info[1], info[2], info[3])
                accumulated_distance = info[6]
            image_info_list.append(info)

    df = pd.DataFrame(image_info_list, columns=['Nombre del Archivo', 'Latitud', 'Longitud', 'Altitud', 'Fecha y Hora', 'Distancia (km)', 'Distancia Acumulada (km)'])
    df.sort_values(by=['Fecha y Hora'], inplace=True)
    df.to_excel(excel_path, index=False)
    print("Excel generado con éxito.")


folder_path = "/content/imagenes"
excel_path = "/content/informacion_imagenes.xlsx"
initial_accumulated_distance = float(input("Ingrese el valor inicial de distancia acumulada: "))
create_excel_with_image_info(folder_path, excel_path, initial_accumulated_distance)

!ls

**MARCADO DE FOTOGRAFIAS** ✏

1.   Coordenadas geográficas
2.   Fecha y hora de captura
3.   Referencia kilométrica






In [ ]:
import os
from PIL import Image, ImageDraw, ImageFont
from datetime import datetime
import exifread
import zipfile
import piexif
import pandas as pd

def get_coordinates(image_path):
    with open(image_path, 'rb') as f:
        exif_tags = exifread.process_file(f)
        if 'GPS GPSLatitude' in exif_tags and 'GPS GPSLongitude' in exif_tags:
            latitude = exif_tags['GPS GPSLatitude']
            longitude = exif_tags['GPS GPSLongitude']
            lat_deg = float(latitude.values[0].num) / float(latitude.values[0].den)
            lat_min = float(latitude.values[1].num) / float(latitude.values[1].den)
            lat_sec = float(latitude.values[2].num) / float(latitude.values[2].den)
            latitude_dec = lat_deg + (lat_min / 60.0) + (lat_sec / 3600.0)
            lon_deg = float(longitude.values[0].num) / float(longitude.values[0].den)
            lon_min = float(longitude.values[1].num) / float(longitude.values[1].den)
            lon_sec = float(longitude.values[2].num) / float(longitude.values[2].den)
            longitude_dec = lon_deg + (lon_min / 60.0) + (lon_sec / 3600.0)
            lat_dir = exif_tags['GPS GPSLatitudeRef'].printable
            lon_dir = exif_tags['GPS GPSLongitudeRef'].printable
            return latitude_dec, lat_dir, longitude_dec, lon_dir
        else:
            return None

def add_telemetry(image_path, df):
    img = Image.open(image_path)
    draw = ImageDraw.Draw(img)
    coordinates = get_coordinates(image_path)
    if coordinates:
        latitude_dec, lat_dir, longitude_dec, lon_dir = coordinates
        font_size_coords = 30
        font_path = "/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf"
        font_coords = ImageFont.truetype(font_path, size=font_size_coords)
        latitud = f"Lat: {latitude_dec:.5f} {lat_dir}"
        longitud = f"Long: {longitude_dec:.5f} {lon_dir}"
        position_lat = (50, img.height - font_size_coords*2.5)
        position_long = (50, img.height - font_size_coords*1.35)
        text_color = (255, 255, 255)
        draw.text(position_lat, latitud, fill=text_color, font=font_coords)
        draw.text(position_long, longitud, fill=text_color, font=font_coords)


    exif_data = img._getexif()
    if exif_data:
        if 36867 in exif_data:
            capture_datetime = exif_data[36867]
            capture_datetime = datetime.strptime(capture_datetime, "%Y:%m:%d %H:%M:%S")
            capture_datetime_str = capture_datetime.strftime("%Y-%m-%d %H:%M:%S")
            font_size_datetime = 30
            font_datetime = ImageFont.truetype(font_path, size=font_size_datetime)
            text_bbox = draw.textbbox((0, 0), capture_datetime_str, font=font_datetime)
            position_datetime = (img.width - text_bbox[2] - 50, img.height - text_bbox[3] - 50)
            draw.text(position_datetime, capture_datetime_str, fill=text_color, font=font_datetime)


    font_size_distance = 40
    font_distance = ImageFont.truetype(font_path, size=font_size_distance)
    distance = df[df['Nombre del Archivo'] == os.path.basename(image_path)]['Distancia Acumulada (km)'].iloc[0]


    integer_part = int(distance)
    decimal_part = int((distance - integer_part) * 1000)  # Multiplicar por 1000 para obtener la parte decimal en metros
    distance_str = f"PKM {integer_part}+{decimal_part:03d}"  # Formatear la cadena con tres dígitos decimales
    text_bbox = draw.textbbox((0, 0), distance_str, font=font_distance)
    position_distance = ((img.width - text_bbox[2]) // 2, img.height - text_bbox[3] - 20)
    draw.text(position_distance, distance_str, fill=text_color, font=font_distance)


    output_path = os.path.join("/content", "imagenes_con_telemetria", os.path.basename(image_path))
    img.save(output_path, exif=img.info.get("exif", b""))
    return output_path

folder_path = "/content/imagenes"
output_folder_path = "/content/imagenes_con_telemetria"
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

processed_images = []
excel_path = "/content/informacion_imagenes.xlsx"
df = pd.read_excel(excel_path)
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        image_path = os.path.join(folder_path, filename)
        processed_image = add_telemetry(image_path, df)
        processed_images.append(processed_image)

with zipfile.ZipFile('/content/imagenes_con_telemetria.zip', 'w') as zipf:
    for image_path in processed_images:
        zipf.write(image_path, arcname=os.path.basename(image_path))


**GENERAR UN MAPA GIS CON LAS IMAGENES ETIQUETADAS**

In [ ]:
import os
import exifread
import folium
from folium import IFrame
import base64
import pandas as pd

def add_image_marker(map_obj, image_path, popup_text):
    # Obtener las coordenadas GPS de la imagen
    tags = exifread.process_file(open(image_path, 'rb'))
    latitude = tags.get('GPS GPSLatitude')
    longitude = tags.get('GPS GPSLongitude')
    if latitude and longitude:
        lat_deg = latitude.values[0].num / latitude.values[0].den
        lat_min = latitude.values[1].num / latitude.values[1].den
        lat_sec = latitude.values[2].num / latitude.values[2].den
        lat_ref = tags.get('GPS GPSLatitudeRef').values
        latitude = lat_deg + lat_min / 60.0 + lat_sec / 3600.0
        if lat_ref == 'S':
            latitude *= -1

        lon_deg = longitude.values[0].num / longitude.values[0].den
        lon_min = longitude.values[1].num / longitude.values[1].den
        lon_sec = longitude.values[2].num / longitude.values[2].den
        lon_ref = tags.get('GPS GPSLongitudeRef').values
        longitude = lon_deg + lon_min / 60.0 + lon_sec / 3600.0
        if lon_ref == 'W':
            longitude *= -1

        with open(image_path, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode('utf-8')


        popup_html = f'<img src="data:image/jpeg;base64,{encoded_string}" width="500px"><br>{popup_text}'
        popup = folium.Popup(IFrame(html=popup_html, width=480, height=360), max_width=2650)
        folium.Marker(location=[latitude, longitude], popup=popup).add_to(map_obj)

def display_images_on_map(folder_path, df):

    coordinates = []

    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        if os.path.isfile(image_path):
            tags = exifread.process_file(open(image_path, 'rb'))
            if tags:
                latitude = tags.get('GPS GPSLatitude')
                longitude = tags.get('GPS GPSLongitude')
                if latitude and longitude:
                    lat_deg = latitude.values[0].num / latitude.values[0].den
                    lat_min = latitude.values[1].num / latitude.values[1].den
                    lat_sec = latitude.values[2].num / latitude.values[2].den
                    lat_ref = tags.get('GPS GPSLatitudeRef').values
                    latitude = lat_deg + lat_min / 60.0 + lat_sec / 3600.0
                    if lat_ref == 'S':
                        latitude *= -1

                    lon_deg = longitude.values[0].num / longitude.values[0].den
                    lon_min = longitude.values[1].num / longitude.values[1].den
                    lon_sec = longitude.values[2].num / longitude.values[2].den
                    lon_ref = tags.get('GPS GPSLongitudeRef').values
                    longitude = lon_deg + lon_min / 60.0 + lon_sec / 3600.0
                    if lon_ref == 'W':
                        longitude *= -1

                    coordinates.append([latitude, longitude])

    if coordinates:
        centroid = [sum(x[0] for x in coordinates) / len(coordinates), sum(x[1] for x in coordinates) / len(coordinates)]
    else:
        centroid = [0, 0]

    map_obj = folium.Map(location=centroid)

    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        if os.path.isfile(image_path):
            tags = exifread.process_file(open(image_path, 'rb'))
            if tags:
                # Obtener el texto del popup con información relevante de la imagen
                popup_text = f"Nombre de archivo: {filename}<br>"
                # Obtener la información de PKM del DataFrame
                pkm_info = df[df['Nombre del Archivo'] == filename]
                if not pkm_info.empty:
                    distance = pkm_info['Distancia Acumulada (km)'].iloc[0]
                    # Formatear el valor de PKM
                    integer_part = int(distance)
                    decimal_part = int((distance - integer_part) * 1000)
                    distance_formatted = f"{integer_part}+{decimal_part:03d}"
                    popup_text += f"PKM: {distance_formatted}<br>"
                add_image_marker(map_obj, image_path, popup_text)


    map_obj.save("mapa_con_imagenes.html")
    return map_obj

folder_path = "/content/imagenes_con_telemetria"
excel_path = "/content/informacion_imagenes.xlsx"
df = pd.read_excel(excel_path)
display_images_on_map(folder_path, df)

**CREANDO UN KMZ CON IMAGENES EMBEBIDAS**

In [ ]:
import os
import base64

def extract_image_info(image_path):
    with open(image_path, 'rb') as f:
        tags = exifread.process_file(f)
        if not tags:
            return None, None, None
        else:
            latitude = tags.get('GPS GPSLatitude')
            longitude = tags.get('GPS GPSLongitude')
            image_name = os.path.basename(image_path)

            if latitude and longitude:

                lat_degrees = latitude.values[0].num / latitude.values[0].den
                lat_minutes = latitude.values[1].num / latitude.values[1].den
                lat_seconds = latitude.values[2].num / latitude.values[2].den
                latitude_dec = lat_degrees + (lat_minutes / 60.0) + (lat_seconds / 3600.0)

                lon_degrees = longitude.values[0].num / longitude.values[0].den
                lon_minutes = longitude.values[1].num / longitude.values[1].den
                lon_seconds = longitude.values[2].num / longitude.values[2].den
                longitude_dec = lon_degrees + (lon_minutes / 60.0) + (lon_seconds / 3600.0)


                lat_dir = tags.get('GPS GPSLatitudeRef').printable
                lon_dir = tags.get('GPS GPSLongitudeRef').printable

                if lat_dir == 'S':
                    latitude_dec *= -1
                if lon_dir == 'W':
                    longitude_dec *= -1
            else:
                latitude_dec, longitude_dec = None, None

            return latitude_dec, longitude_dec, image_name

def generate_kml_with_images(image_info_list):
    placemark_template = """<Placemark>
    <name>{}</name>
    <description><![CDATA[<img src="data:image/jpeg;base64,{}" height="300" width="400"/>]]></description>
    <Point>
        <coordinates>{},{},0</coordinates>
    </Point>
</Placemark>
"""
    kml_content = """<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
<Document>
"""

    for image_info in image_info_list:
        latitude, longitude, image_name = image_info
        with open(os.path.join(folder_path, image_name), 'rb') as f:
            encoded_image = base64.b64encode(f.read()).decode('utf-8')
        kml_content += placemark_template.format(image_name, encoded_image, longitude, latitude)

    kml_content += """</Document>
</kml>"""

    return kml_content

def generate_kmz(folder_path, kmz_path):
    image_info_list = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
            image_path = os.path.join(folder_path, filename)
            latitude, longitude, image_name = extract_image_info(image_path)
            image_info_list.append((latitude, longitude, image_name))

    kml_content = generate_kml_with_images(image_info_list)

    kml_file = os.path.join(folder_path, "images.kml")
    with open(kml_file, "w") as f:
        f.write(kml_content)

    import zipfile
    with zipfile.ZipFile(kmz_path, 'w') as kmz:
        kmz.write(kml_file, arcname=os.path.basename(kml_file))

folder_path = "/content/imagenes_con_telemetria"

kmz_path = "/content/images.kmz"

generate_kmz(folder_path, kmz_path)


**ELIMINAR LA INFORMACION CARGADA EN IMAGENES**

In [3]:
!rm -rf /content/imagenes
!rm -rf /content/imagenes_con_telemetria


Fin del codigo.